[출처](https://www.youtube.com/watch?v=b4Vyma9wPHo&list=PLlMkM4tgfjnJ3I-dbhO9JTw7gNty6o_2m&index=3)

In [29]:
import numpy as np
from torch import nn, optim, cuda
import torch
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
Y = np.array([1, 0, 0])

In [3]:
Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])

In [4]:
print(np.sum(-Y * np.log(Y_pred1)))
print(np.sum(-Y * np.log(Y_pred2)))

0.35667494393873245
2.3025850929940455


In [15]:
loss = nn.CrossEntropyLoss()

In [14]:
Y = Variable(torch.LongTensor([0]), requires_grad = False)

In [18]:
Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

In [19]:
l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

In [20]:
print(l1.data)
print(l2.data)

tensor(0.4170)
tensor(1.8406)


In [42]:
from torchvision import datasets, transforms
batch_size = 64
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())


In [43]:
from torch.utils import data
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [44]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [45]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

Training MNIST Model on cpu


In [46]:
model = Net()

In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr= 0.01, momentum = 0.5)

In [48]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [56]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [57]:
import time
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 0.234990
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 0.314423
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 0.171069
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 0.289621
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 0.261543
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 0.382402
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 0.117387
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 0.206412
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 0.217469
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 0.222531
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 0.230479
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 0.194943
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 0.352316
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 0.254049
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 0.299973
Train Epoch: 1 | Batch Status: 9600/60